### Setup inicial

In [ ]:
# Librerías básicas
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# Librerías BN
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import HillClimbSearch, BIC, K2, BDeu
from pgmpy.estimators import MaximumLikelihoodEstimator

# Configuración general
pd.set_option("display.max_columns", None)
plt.rcParams["figure.figsize"] = (8, 6)

print("✅ Librerías cargadas")


### Cargar y preparar datos

In [ ]:
# Cargar dataset
data = pd.read_csv("tv_bn_dataset.csv")

# Convertir todas las variables a categóricas (importante para BN)
for c in data.columns:
    data[c] = data[c].astype("category")

print("✅ Datos cargados y convertidos a categóricos")
display(data.head())


### Comparar métodos de scoring

#### 1. BIC

In [ ]:
hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method=BIC(data))
print("✅ Modelo aprendido con BIC")

model = DiscreteBayesianNetwork(best_model.edges())
model.fit(data, estimator=MaximumLikelihoodEstimator)
print("✅ Modelo entrenado con BIC")

# Convertir a un grafo de networkx y visualizar la estructura del modelo
G = nx.DiGraph()
G.add_nodes_from(model.nodes())
G.add_edges_from(model.edges())

plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_size=1000, node_color="lightblue", font_size=10, font_weight="bold", arrowsize=20)
plt.title("BIC")
plt.show()



#### 2. K2

In [ ]:
hc = HillClimbSearch(data)
best_model_K2 = hc.estimate(scoring_method=K2(data))
print("✅ Modelo aprendido con K2")


model_K2 = DiscreteBayesianNetwork(best_model_K2.edges())
model_K2.fit(data, estimator=MaximumLikelihoodEstimator)
print("✅ Modelo entrenado con K2")

# Convertir a un grafo de networkx y visualizar la estructura del modelo
G_K2 = nx.DiGraph()
G_K2.add_nodes_from(model_K2.nodes())
G_K2.add_edges_from(model_K2.edges())

plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G_K2, seed=42)
nx.draw(G_K2, pos, with_labels=True, node_size=1000, node_color="lightblue", font_size=10, font_weight="bold", arrowsize=20)
plt.title("K2")
plt.show()



#### 3. BDeu

In [ ]:
ess_list = [1, 5, 10, 50, 100, 500]

for ess in ess_list:
    print(f"--- equivalent_sample_size = {ess} ---")
    best_model_BDeu = hc.estimate(scoring_method=BDeu(data, equivalent_sample_size=ess))
    print("Edges:", list(best_model_BDeu.edges()))
    
    model_BDeu = DiscreteBayesianNetwork(best_model_BDeu.edges())
    model_BDeu.fit(data, estimator=MaximumLikelihoodEstimator)
    
    # Grafo y visualización (usa pos si ya existe para mantener la disposición)
    G_BDeu = nx.DiGraph()
    G_BDeu.add_nodes_from(model_BDeu.nodes())
    G_BDeu.add_edges_from(model_BDeu.edges())

    plt.figure(figsize=(10, 8))
    layout = pos if 'pos' in globals() else nx.spring_layout(G_BDeu, seed=42)
    nx.draw(G_BDeu, layout, with_labels=True, node_size=1000, node_color="lightblue",
            font_size=10, font_weight="bold", arrowsize=20)
    plt.title(f"BDeu (ess={ess})")
    plt.show()


